In [1]:
import pandas as pd 
import numpy as np
from sklearn import preprocessing
data=pd.read_csv("final_data.csv")
#final_data is the final pre processed data

In [2]:
data.head()

,Operator,In Out Travelling,Network Type,Rating,Call Drop Category,Latitude,Longitude,State Name,Average Data Speed Download,Average Data Speed Upload,Population,Literacy Rate,Rainfall,Area of the state,Average_Signal
0,Airtel,Indoor,4G,3,Satisfactory,0.196228,0.325356,Karnataka,0.271480,0.197512,2,7,3,0.504087,0.476721
1,Vodafone,Indoor,4G,1,Call Dropped,0.632907,0.135432,Gujarat,0.211519,0.370879,2,8,2,0.518040,0.477252
2,Airtel,Indoor,3G,1,Call Dropped,0.127093,0.290671,Tamil Nadu,0.036321,0.062303,2,8,2,0.300607,0.478454
3,Airtel,Indoor,4G,3,Satisfactory,0.580181,0.155327,Gujarat,0.175517,0.191200,2,8,2,0.518040,0.477699
4,Airtel,Travelling,4G,3,Satisfactory,0.425737,0.180503,Maharashtra,0.333021,0.395612,4,8,4,0.886194,0.476859


In [3]:
data['Operator'].unique()

array(['Airtel', 'Vodafone', 'RJio', 'BSNL', 'Idea'], dtype=object)

In [4]:
RJio=data[data['Operator']=='RJio']

In [5]:
RJio.count()

Operator                       15218
In Out Travelling              15218
Network Type                   15218
Rating                         15218
Call Drop Category             15218
Latitude                       15218
Longitude                      15218
State Name                     15218
Average Data Speed Download    15218
Average Data Speed Upload      15218
Population                     15218
Literacy Rate                  15218
Rainfall                       15218
Area of the state              15218
Average_Signal                 15218
dtype: int64

In [6]:
from sklearn import preprocessing
from numpy.linalg import inv
from sklearn.linear_model import LinearRegression

In [7]:
X=RJio[['Latitude','Longitude']]

In [8]:
X.count()

Latitude     15218
Longitude    15218
dtype: int64

In [9]:
y=RJio['Average Data Speed Download']

In [10]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) 
x_train.reset_index(inplace = True, drop = True) 
x_test.reset_index(inplace = True, drop = True) 
y_train.reset_index(inplace = True, drop = True) 
y_test.reset_index(inplace = True, drop = True) 
#We have used holdout method and 70%of data is taken as training data and rest as testing 

In [11]:
x_train.head()

,Latitude,Longitude
0,0.840193,0.300109
1,0.877370,0.311599
2,0.838645,0.303074
3,0.857339,0.300618
4,0.839376,0.312661


In [12]:
n=x_train.count()[0]

In [13]:
n

10652

# Download Data Speed Latitude Longitude

In [14]:
sum_x1=0
sum_x2=0
sum_y1=0
sum_x1x1=0
sum_x1x2=0
sum_x2x2=0
sum_x1y1=0
sum_x2y1=0
for i in range(0,n):
    sum_x1=sum_x1+x_train['Latitude'][i]
    sum_x2=sum_x2+x_train['Longitude'][i]
    sum_y1=sum_y1+y_train[i]
    sum_x1x1=sum_x1x1+x_train['Latitude'][i]*x_train['Latitude'][i]
    sum_x1x2=sum_x1x2+x_train['Latitude'][i]*x_train['Longitude'][i]
    sum_x2x2=sum_x2x2+x_train['Longitude'][i]*x_train['Longitude'][i]
    sum_x1y1=sum_x1y1+x_train['Latitude'][i]*y_train[i]
    sum_x2y1=sum_x2y1+x_train['Longitude'][i]*y_train[i]
A = np.array([[n, sum_x1, sum_x2], [sum_x1, sum_x1x1, sum_x1x2],[sum_x2,sum_x1x2,sum_x2x2]])
B=np.array([sum_y1, sum_x1y1, sum_x2y1])

In [15]:
A

array([[10652.        ,  5672.97885046,  3734.62832884],
       [ 5672.97885046,  3401.94950326,  2068.35224805],
       [ 3734.62832884,  2068.35224805,  1716.35468993]])

In [16]:
B

array([7133.78336463, 3805.79885032, 2651.48197597])

In [17]:
Inverse=inv(A)
w0=Inverse[0][0]*B[0]+Inverse[0][1]*B[1]+Inverse[0][2]*B[2]
w1=Inverse[1][0]*B[0]+Inverse[1][1]*B[1]+Inverse[1][2]*B[2]
w2=Inverse[2][0]*B[0]+Inverse[2][1]*B[1]+Inverse[2][2]*B[2]
W=np.array([w0,w1,w2])

In [18]:
W

array([ 0.56916551, -0.0624263 ,  0.38161106])

In [19]:
y_pred=w0+w1*x_test['Latitude']+w2*x_test['Longitude']
#y_pred gives the predicted values of y based on the regression model
MSE=0
for i in range(0,len(y_test)):
    MSE=MSE+(y_pred[i]-y_test[i])**2
RMSE=np.sqrt(MSE/len(y_test))
RMSE
#RMSE is used to find the goodness of the model and is calacuated as root of sum of square of difference of actual and prediacted values.

0.1855324495973508

In [20]:
ybar=y_test.mean()
ybar

0.6672640139467776

In [22]:
SSR=0
SST=0
SSE=0
y_test.count()
#SST is total sum of squares
#SSR is sum of squares due to regression
#SSE is sum of squares due to error  

4566

In [23]:
for i in range (0,len(y_test)):
    SST=SST+(y_test[i]-ybar)*(y_test[i]-ybar)
    SSR=SSR+(y_pred[i]-ybar)*(y_pred[i]-ybar)
    SSE=SSE+(y_test[i]-y_pred[i])*(y_test[i]-y_pred[i])  

In [24]:
print("Total sum of squares=", SST )

Total sum of squares= 179.66258072187546


In [25]:
print("Sum of squares due to regression =", SSR)

Sum of squares due to regression = 24.536165214076945


In [26]:
print("Sum of square due to error=", SSE)

Sum of square due to error= 157.172175471508


In [27]:
Rsquare=SSR/SST

In [28]:
p=2
n=len(y_test)
len(x_train)+len(x_test)

15218

Significance Test

F Test


In [29]:
W

array([ 0.56916551, -0.0624263 ,  0.38161106])

In [30]:
w0=W[0]
w0

0.5691655072742914

In [31]:
w1=W[1]
w2=W[2]

Regression line is y= b0 + b1 * x1 + b2 * x2

b1 is coefficient of x1

b2 is coefficient of x2

H0 : b1=b2=0

Ha : One or both parameters not equal to zero

In [32]:
MSR=SSR/2
MSR

12.268082607038473

In [33]:
MSE=SSE/(len(y_test)-3)
MSE

0.034444921207869385

In [34]:
F=MSR/MSE

In [35]:
F

356.165210336892

In [36]:
len(y_test)-3

4563

F alpha with numerator dof=2 and denominator dof=n-3(=4563) at level of significance=0.05 is equal to 2.998

In [37]:
Falpha=2.99769991

In [41]:
F=MSR/MSE
#The F test is used to determine whether a significant relationship exists between the dependent variable and the set of all the independent variables; hence F test are referred to as  the test for overall significance

So we reject H0. Now for b1 and b2 t tests are performed separately.


T test for b1

H0 : b1=0

Ha : b1 is not equal to zero

In [42]:
sum=0
m=x_test['Latitude'].mean()
for i in range(0,len(x_test)):
    sum=sum+((x_test['Latitude'][i]-m)**2)
sum

169.30069409856878

In [43]:
import math
x=math.sqrt(sum)
z=SSE/(len(y_test)-3)
s=math.sqrt(z)
sb1=s/x

In [44]:
t=w1/sb1

In [48]:
t
#t distribution with n − p − 1 degrees of freedom.
#The t test is used to determine whether each of the individual independent variables is significant. A separate t test is conducted for each of the independent variables in the model; hence t tests are called as the test for individual significance. 

-4.3765750113948245

In [49]:
len(y_test)-3

4563

So talpha=2.242 at dof=len(y_test)-3 and alpha=0.025

In [50]:
talpha=2.242143
t<=-talpha

True

Hence we reject null hypothesis

T Test for b2

H0 : b2=0

Ha : b2 not equal to zero

In [51]:
sum=0
m=x_test['Longitude'].mean()
for i in range(0,len(x_test)):
    sum=sum+((x_test['Longitude'][i]-m)**2)
x=math.sqrt(sum)
z=SSE/(len(y_test)-3)
s=math.sqrt(z)
sb2=s/x

In [52]:
t=w2/sb2

In [53]:
t

27.178953212009333

In [54]:
t>=talpha

True

Hence we reject null hypothesis

# Hence we can conclude that Download data speed does depend on Latitude and Longitude significantly for RJio

# Upload Data Speed Latitude Longitude

Now doing the same for Data Speed Upload

In [55]:
n=x_train.count()[0]
y=RJio['Average Data Speed Upload']
X=RJio[['Latitude','Longitude']]

In [56]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) 
x_train.reset_index(inplace = True, drop = True) 
x_test.reset_index(inplace = True, drop = True) 
y_train.reset_index(inplace = True, drop = True) 
y_test.reset_index(inplace = True, drop = True) 

In [57]:
sum_x1=0
sum_x2=0
sum_y1=0
sum_x1x1=0
sum_x1x2=0
sum_x2x2=0
sum_x1y1=0
sum_x2y1=0
for i in range(0,n):
    sum_x1=sum_x1+x_train['Latitude'][i]
    sum_x2=sum_x2+x_train['Longitude'][i]
    sum_y1=sum_y1+y_train[i]
    sum_x1x1=sum_x1x1+x_train['Latitude'][i]*x_train['Latitude'][i]
    sum_x1x2=sum_x1x2+x_train['Latitude'][i]*x_train['Longitude'][i]
    sum_x2x2=sum_x2x2+x_train['Longitude'][i]*x_train['Longitude'][i]
    sum_x1y1=sum_x1y1+x_train['Latitude'][i]*y_train[i]
    sum_x2y1=sum_x2y1+x_train['Longitude'][i]*y_train[i]
A = np.array([[n, sum_x1, sum_x2], [sum_x1, sum_x1x1, sum_x1x2],[sum_x2,sum_x1x2,sum_x2x2]])
B=np.array([sum_y1, sum_x1y1, sum_x2y1])
Inverse=inv(A)
w0=Inverse[0][0]*B[0]+Inverse[0][1]*B[1]+Inverse[0][2]*B[2]
w1=Inverse[1][0]*B[0]+Inverse[1][1]*B[1]+Inverse[1][2]*B[2]
w2=Inverse[2][0]*B[0]+Inverse[2][1]*B[1]+Inverse[2][2]*B[2]
W=np.array([w0,w1,w2])

In [58]:
W


array([ 0.42705896, -0.21945917,  0.25481481])

In [59]:
y_pred=w0+w1*x_test['Latitude']+w2*x_test['Longitude']
#y_pred gives the predicted values of y based on the regression model
MSE=0
for i in range(0,len(y_test)):
    MSE=MSE+(y_pred[i]-y_test[i])**2
RMSE=np.sqrt(MSE/len(y_test))
RMSE
#RMSE is used to find the goodness of the model and is calacuated as root of sum of square of difference of actual and prediacted values.

0.12583230294661485

In [61]:
ybar=y_test.mean()
SSR=0
SST=0
SSE=0
for i in range (0,len(y_test)):
    SST=SST+(y_test[i]-ybar)*(y_test[i]-ybar)
    SSR=SSR+(y_pred[i]-ybar)*(y_pred[i]-ybar)
    SSE=SSE+(y_test[i]-y_pred[i])*(y_test[i]-y_pred[i])
print("Total sum of squares=", SST )
print("Sum of squares due to regression =", SSR)
print("Sum of square due to error=", SSE)

Total sum of squares= 85.2620194205804
Sum of squares due to regression = 15.721192390280583
Sum of square due to error= 72.29698681049895


In [62]:
w0

0.4270589573416661

In [63]:
w1

-0.21945916872573934

In [64]:
w2

0.25481480611764606

So linear regression line is y=b0 + b1 * x1 + b2 * x2

b1 is coefficient of x1


b2 is coefficient of x2

H0 : b1=b2=0


Ha : One or both parameters not equal to zero

In [65]:
MSR=SSR/2
MSE=SSE/(len(y_test)-3)
F=MSR/MSE
Falpha=2.99769991
F

496.11888435185

In [66]:
F>=Falpha

True

Hence we reject H0. As both parameters are not equal to zero so we will find out individually about each parameter through t test.

T test for b1

H0 : b1=0

Ha : b1 is not equal to zero

In [67]:
sum=0
m=x_test['Latitude'].mean()
for i in range(0,len(x_test)):
    sum=sum+((x_test['Latitude'][i]-m)**2)
sum

169.30069409856878

In [68]:
x=math.sqrt(sum)
z=SSE/(len(y_test)-3)
s=math.sqrt(z)
sb1=s/x

In [69]:
sb1

0.009673986898429241

In [70]:
t= w1/sb1

In [71]:
t

-22.685493688375036

talpha at dof = n-2 at 0.05/2=0.25 level of significance is 2.24

In [72]:
talpha=2.242143

In [73]:
t<=-talpha

True

Hence we reject the null hypothesis.

T test for b2

H0 : b2=0

Ha : b2 is not equal to zero

In [74]:
sum=0
m=x_test['Longitude'].mean()
for i in range(0,len(x_test)):
    sum=sum+((x_test['Longitude'][i]-m)**2)
x=math.sqrt(sum)
z=SSE/(len(y_test)-3)
s=math.sqrt(z)
sb2=s/x

In [75]:
sb2

0.009522708524968398

In [76]:
t= w2/s

In [77]:
t

2.0243695448903973

In [78]:
t>=talpha

False

Hence we accept the null hypothesis.

# Hence we can conclude that Data Speed Upload does depend on Latitude but not on Longitude significantly for RJio.

# Signal Strength Latitude Longitude

In [80]:
n=x_train.count()[0]
y=RJio['Average_Signal']
X=RJio[['Latitude','Longitude']]
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) 
x_train.reset_index(inplace = True, drop = True) 
x_test.reset_index(inplace = True, drop = True) 
y_train.reset_index(inplace = True, drop = True) 
y_test.reset_index(inplace = True, drop = True) 
sum_x1=0
sum_x2=0
sum_y1=0
sum_x1x1=0
sum_x1x2=0
sum_x2x2=0
sum_x1y1=0
sum_x2y1=0
for i in range(0,n):
    sum_x1=sum_x1+x_train['Latitude'][i]
    sum_x2=sum_x2+x_train['Longitude'][i]
    sum_y1=sum_y1+y_train[i]
    sum_x1x1=sum_x1x1+x_train['Latitude'][i]*x_train['Latitude'][i]
    sum_x1x2=sum_x1x2+x_train['Latitude'][i]*x_train['Longitude'][i]
    sum_x2x2=sum_x2x2+x_train['Longitude'][i]*x_train['Longitude'][i]
    sum_x1y1=sum_x1y1+x_train['Latitude'][i]*y_train[i]
    sum_x2y1=sum_x2y1+x_train['Longitude'][i]*y_train[i]
A = np.array([[n, sum_x1, sum_x2], [sum_x1, sum_x1x1, sum_x1x2],[sum_x2,sum_x1x2,sum_x2x2]])
B=np.array([sum_y1, sum_x1y1, sum_x2y1])
Inverse=inv(A)
w0=Inverse[0][0]*B[0]+Inverse[0][1]*B[1]+Inverse[0][2]*B[2]
w1=Inverse[1][0]*B[0]+Inverse[1][1]*B[1]+Inverse[1][2]*B[2]
w2=Inverse[2][0]*B[0]+Inverse[2][1]*B[1]+Inverse[2][2]*B[2]
W=np.array([w0,w1,w2])
W

array([ 4.77867357e-01, -1.60421250e-04,  1.71528656e-03])

In [81]:
y_pred=w0+w1*x_test['Latitude']+w2*x_test['Longitude']
MSE=0
y_pred=[]
MSE=0
for i in range(0,len(y_test)):
    y_pred.append(w0+w1*x_test['Latitude'][i]+w2*x_test['Longitude'][i])
    MSE=MSE+(y_pred[i]-y_test[i])**2
RMSE=np.sqrt(MSE/len(y_test))
RMSE

0.000841722276378673

In [83]:
ybar=y_test.mean()
SSR=0
SST=0
SSE=0
for i in range (0,len(y_test)):
    SST=SST+(y_test[i]-ybar)*(y_test[i]-ybar)
    SSR=SSR+(y_pred[i]-ybar)*(y_pred[i]-ybar)
    SSE=SSE+(y_test[i]-y_pred[i])*(y_test[i]-y_pred[i])
print("Total sum of squares=", SST )
print("Sum of squares due to regression =", SSR)
print("Sum of square due to error=", SSE)

Total sum of squares= 0.0036553825853142085
Sum of squares due to regression = 0.0005020608671729043
Sum of square due to error= 0.0032349945192608667


So linear regression line is y=b0 + b1 * x1 + b2 * x2

F test

H0 : b1=b2=0


Ha : One or both parameters not equal to zero

In [84]:
MSR=SSR/2
MSE=SSE/(len(y_test)-3)
F=MSR/MSE
Falpha=2.99769991

In [85]:
F

354.0815484029613

In [86]:
F>=Falpha

True

Hence we reject H0. As both parameters are not equal to zero so we will find out individually about each parameter through t test.

T test for b1

H0 : b1=0


Ha : b1 is not equal to zero


In [87]:
sum=0
m=x_test['Latitude'].mean()
for i in range(0,len(x_test)):
    sum=sum+((x_test['Latitude'][i]-m)**2)
sum

169.30069409856878

In [88]:
x=math.sqrt(sum)
z=SSE/(len(y_test)-3)
s=math.sqrt(z)
sb1=s/x

In [89]:
sb1

6.471160491482031e-05

In [90]:
t=w1/sb1

In [91]:
t

-2.479018262460421

In [92]:
w1

-0.0001604212503769631

In [93]:
t<=-talpha

True

Hence we reject null hypothesis

T test for b2

H0 : b2=0

Ha : b2 is not equal to zero

In [94]:
sum=0
m=x_test['Longitude'].mean()
for i in range(0,len(x_test)):
    sum=sum+((x_test['Longitude'][i]-m)**2)
x=math.sqrt(sum)
z=SSE/(len(y_test)-3)
s=math.sqrt(z)
sb2=s/x
t=w2/sb2

In [95]:
t

26.92771598871534

In [96]:
t>=talpha

True

Hence we reject null hypothesis

# Hence we can conclude that Signal Strength does depend on Latitude and Longitude significantly for RJio.

In [97]:
#For download data speed there does exist a significant relationship with latitude and longitude.
#For upload data speed there exists a significant relationship with latitude only.
#For signal strength also there exists a significant relationship with latitude and longitude.
